In [1]:
import numpy as np
import rasterio
import pandas as pd
import os
import sys

In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)



In [3]:
from constants import SERVER_PATH, OUTPUT_PATH, SIMULATION_FEATURES_DIR


In [4]:
SIMULATION_FEATURES_DIR

['/Users/romero61/../../capstone/pyforest/ml_data/output/sim_lup_features']

In [5]:
#output- update this for subsequent runs
output_folder = os.path.join(OUTPUT_PATH[0], 'predictions-log-lut-areas')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


In [6]:
sim25_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim25', "sim25_raster.tif"))
sim50_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim50', "sim50_raster.tif"))
sim5_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim5', "sim5_raster.tif"))
simhedges_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'simhedges', "simhedges_raster.tif"))


In [7]:
sim25_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-25-all-feature-predicition.tiff')
sim50_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-50-all-feature-predicition.tiff')
sim5_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-5-all-feature-predicition.tiff')
simhedges_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-hedges-all-feature-predicition.tiff')


In [10]:
# Open the land use raster file
with rasterio.open(sim25_lup) as src:
    land_use_array = src.read(1)
    p1 = src.profile

# Open the deforestation probability raster file
with rasterio.open(sim25_lup_prediction) as src:
    deforestation_probability_array = src.read(1)
    p2 = src.profile
print(p1)
print(p2)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -1.0, 'width': 20381, 'height': 22512, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, -62.64186038139295,
       0.0, -0.00026949458523585647, -19.287457970745013), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -1.0, 'width': 20381, 'height': 22512, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, -62.64186038139295,
       0.0, -0.00026949458523585647, -19.287457970745013), 'tiled': False, 'interleave': 'band'}


# 25

In [11]:
# Open the land use raster file
with rasterio.open(sim25_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim25_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 838.045  # 28.94785483442850804 m * 28.94785483442850804 m = 838.045 square meters EPSG:32721
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 838.045  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-25-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)


Area of land use type 0: 0.0 square meters
Area of land use type 1: 12236239734.029999 square meters
Area of land use type 2: 23938267682.809998 square meters
Area of land use type 3: 54972357917.75 square meters
Area of land use type 4: 850029881.545 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 1223623.9734029998 hectares
Area of land use type 2: 2393826.7682809997 hectares
Area of land use type 3: 5497235.791775 hectares
Area of land use type 4: 85002.9881545 hectares


# 50%

In [13]:
# Open the land use raster file
with rasterio.open(sim50_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim50_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 838.045  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 838.045  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-50-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)

Area of land use type 0: 0.0 square meters
Area of land use type 1: 7944978352.74 square meters
Area of land use type 2: 47095170701.149994 square meters
Area of land use type 3: 36111692591.909996 square meters
Area of land use type 4: 850032395.68 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 794497.835274 hectares
Area of land use type 2: 4709517.070114999 hectares
Area of land use type 3: 3611169.2591909994 hectares
Area of land use type 4: 85003.23956799999 hectares


# 5%

In [14]:
# Open the land use raster file
with rasterio.open(sim5_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim5_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 838.045  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 838.045  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-5-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)

Area of land use type 0: 0.0 square meters
Area of land use type 1: 16365350302.404999 square meters
Area of land use type 2: 5118995075.61 square meters
Area of land use type 3: 69652548897.165 square meters
Area of land use type 4: 850018148.915 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 1636535.0302404999 hectares
Area of land use type 2: 511899.50756099995 hectares
Area of land use type 3: 6965254.8897164995 hectares
Area of land use type 4: 85001.81489149999 hectares


# Hedgerows

In [15]:
# Open the land use raster file
with rasterio.open(simhedges_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(simhedges_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
'''This line is creating a masked array from the land_use_array. 
A masked array is a numpy array that has a separate Boolean mask that 
indicates whether each value in the array is valid or not. 
In this case, the masked_less function is used to create a mask 
that is True for all values in the array that are less than 0, 
and False for all other values. This effectively ignores all values less than 0 in the 
subsequent calculations.'''
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 838.045  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 838.045  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-hedges-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)

Area of land use type 0: 0.0 square meters
Area of land use type 1: 20277853893.765 square meters
Area of land use type 2: 2886681200.39 square meters
Area of land use type 3: 67989778784.395 square meters
Area of land use type 4: 850034071.77 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 2027785.3893765 hectares
Area of land use type 2: 288668.120039 hectares
Area of land use type 3: 6798977.8784395 hectares
Area of land use type 4: 85003.407177 hectares
